In [18]:
import pandas as pd
from collections import Counter

def split_and_clean_course_code(l):
    if type(l) != str:
        return []
    tmp = l.split(',')
    return [c.replace('-','').strip()[:6] for c in tmp]

In [19]:
Projections = {
                'Old': ['StudentsAudit_old.xlsx','ID','Audit_Courses_to_add'],
                'New': ['StudentsAudit.xlsx','ID','Audit_Courses_to_add']
}

In [20]:
old = pd.read_excel(Projections['Old'][0])[Projections['Old'][1:]]
new = pd.read_excel(Projections['New'][0])[Projections['New'][1:]]
old.columns = ['ID','old']
new.columns = ['ID','new']
df = pd.merge(old, new, on='ID', how='outer')

count = len(df)
count1 = len(df[df.old.isnull() & df.new.notnull()])
count2 = len(df[df.old.notnull() & df.new.isnull()])
if count1:
    print(f'found {count1} in new but missing from old ({count1/count*100:.2f}%):')
    print(df[df.old.isnull() & df.new.notnull()])
if count2:
    print(f'\nfound {count2} in old but missing from new ({count2/count*100:.2f}%):')
    print(df[df.old.notnull() & df.new.isnull()])

df.old = df.old.apply(split_and_clean_course_code)
df.new = df.new.apply(split_and_clean_course_code)
df['Delta_old_new'] = pd.Series(dtype='object')
df['Delta_new_old'] = pd.Series(dtype='object')
df['Match'] = pd.Series(dtype='float')

for i, r in df.iterrows():
    o = Counter(r.old)
    n = Counter(r.new)
    Delta_o_n = list((o-n).elements())
    Delta_n_o = list((n-o).elements())
    df.at[i,'Delta_old_new'] = Delta_o_n
    df.at[i,'Delta_new_old'] = Delta_n_o
    if len(r.old)+len(r.new) == 0:
        df.at[i,'Match'] = 1
    else:
        df.at[i,'Match'] = 1 - (len(Delta_o_n)+len(Delta_n_o)) / (len(r.old)+len(r.new))

df.to_excel('Compare_Projections.xlsx')
# display(df)

print(f'\ncompared {len(df)} students, with similarity ratio: {df.Match.sum()/len(df)*100:.2f}%')
print(f'\nHere are the differences:')
display(df[ (df.Delta_old_new.map(len)>0) | (df.Delta_new_old.map(len)>0) ])


found 4 in old but missing from new (0.25%):
            ID     old  new
309  202004716  GroupA  NaN
319  202005267  GroupA  NaN
362  202006867  GroupA  NaN
715  202104682  GroupA  NaN

compared 1619 students, with similarity ratio: 99.63%

Here are the differences:


,ID,old,new,Delta_old_new,Delta_new_old,Match
3,201514581,"[SWE320, CIT466, CIT460, SWE321, SEC330, SEC33...","[SWE320, SWE321, CIT466, CIT460, CIT461, SEC330]",[SEC331],[],0.923077
11,201619114,"[SEC335, SEC336, NET351, ARA130, ISL135, NET352]","[SEC335, SEC336, NET351, NET352, ARA130]",[ISL135],[],0.909091
208,201934209,"[SWE225, SEC335, SEC336, NET351, NET352, CIT30...","[SWE225, SEC335, SEC336, NET351, NET352, CIT305]",[CIT461],[],0.923077
280,202003373,"[NET351, CIT460, CIT466, MTH281, NET352, CIT461]","[NET351, NET352, CIT460, CIT461, CIT466]",[MTH281],[],0.909091
309,202004716,[GroupA],[],[GroupA],[],0.000000
319,202005267,[GroupA],[],[GroupA],[],0.000000
362,202006867,[GroupA],[],[GroupA],[],0.000000
537,202019711,"[CIT466, CIT460, GroupD, INS211, INS311, INS46...","[CIT466, CIT460, CIT461, GroupD, INS211, INS311]","[INS467, INS469]",[],0.857143
588,202033742,"[ACC202, CIT365, CIT285, INS311, GroupA]","[ACC202, CIT365, CIT285, INS311, OPR300]",[GroupA],[OPR300],0.800000
715,202104682,[GroupA],[],[GroupA],[],0.000000
